https://mp.weixin.qq.com/s/0qOGevwTltu2SJ-QanW0VA

https://mp.weixin.qq.com/s/kDC5VKm0j2VufIo-aOlfrg

# 工具包导入&数据读取
## 工具包导入

In [16]:
import numpy as np
import pandas as pd
import os
from datetime import date 
from sklearn.metrics import f1_score
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

from dateutil.parser import parse

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

from itertools import product
import ast

from datetime import timedelta 
from time import time

## 数据读取

In [2]:
path = '../input/'

air_visit_data      =  pd.read_csv(path + 'air_visit_data.csv') 

hpg_store_info      =  pd.read_csv(path + 'hpg_store_info.csv')
air_store_info      =  pd.read_csv(path + 'air_store_info.csv')

hpg_reserve         =  pd.read_csv(path + 'hpg_reserve.csv')
air_reserve         =  pd.read_csv(path + 'air_reserve.csv') 

date_info           =  pd.read_csv(path + 'date_info.csv')  

store_id_relation   =  pd.read_csv(path + 'store_id_relation.csv')     

sample_submission   =  pd.read_csv(path + 'sample_submission.csv')

**air_visit_data.csv**

This file contains historical visit data for the air restaurants.

- air_store_id
- visit_date - the date
- visitors - the number of visitors to the restaurant on the date

In [3]:
air_visit_data.head()

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6


**hpg_store_info.csv**

This file contains information about select hpg restaurants. Column names and contents are self-explanatory.

- hpg_store_id
- hpg_genre_name
- hpg_area_name
- latitude
- longitude

Note: latitude and longitude are the latitude and longitude of the area to which the store belongs

In [4]:
hpg_store_info.head()

,hpg_store_id,hpg_genre_name,hpg_area_name,latitude,longitude
0,hpg_6622b62385aec8bf,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
1,hpg_e9e068dd49c5fa00,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
2,hpg_2976f7acb4b3a3bc,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
3,hpg_e51a522e098f024c,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221
4,hpg_e3d0e1519894f275,Japanese style,Tōkyō-to Setagaya-ku Taishidō,35.643675,139.668221


**air_store_info.csv**

This file contains information about select air restaurants. Column names and contents are self-explanatory.

- air_store_id
- air_genre_name
- air_area_name
- latitude
- longitude

Note: latitude and longitude are the latitude and longitude of the area to which the store belongs

In [5]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


**hpg_reserve.csv**

This file contains reservations made in the hpg system.

- hpg_store_id - the restaurant's id in the hpg system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

In [6]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


**air_reserve.csv**

This file contains reservations made in the air system. Note that the reserve_datetime indicates the time when the reservation was created, whereas the visit_datetime is the time in the future where the visit will occur.

- air_store_id - the restaurant's id in the air system
- visit_datetime - the time of the reservation
- reserve_datetime - the time the reservation was made
- reserve_visitors - the number of visitors for that reservation

In [7]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [8]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


**store_id_relation.csv**

This file allows you to join select restaurants that have both the air and hpg system.

- hpg_store_id
- air_store_id

In [9]:
store_id_relation.head()

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809
2,air_c7f78b4f3cba33ff,hpg_cd8ae0d9bbd58ff9
3,air_947eb2cae4f3e8f2,hpg_de24ea49dc25d6b8
4,air_965b2e0cf4119003,hpg_653238a84804d8e7


**sample_submission.csv**

This file shows a submission in the correct format, including the days for which you must forecast.

- id - the id is formed by concatenating the air_store_id and visit_date with an underscore
- visitors - the number of visitors forecasted for the store and date combination

In [10]:
sample_submission.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,0
1,air_00a91d42b08b08d9_2017-04-24,0
2,air_00a91d42b08b08d9_2017-04-25,0
3,air_00a91d42b08b08d9_2017-04-26,0
4,air_00a91d42b08b08d9_2017-04-27,0


## 数据合并

In [11]:
sample_submission['air_store_id'] = sample_submission['id'].apply(lambda x: x[:-11]).values
sample_submission['visit_date'] = sample_submission['id'].apply(lambda x: x[-10:]).values

del sample_submission['id']
gc.collect()

60

In [12]:
df_meta = pd.concat([air_visit_data, sample_submission],
                    axis=0,
                    ignore_index=True)

# 合并date_info
df_meta = df_meta.merge(date_info,
                        left_on=['visit_date'],
                        right_on=['calendar_date'],
                        how='left')
# 合并air_store_info
df_meta = df_meta.merge(air_store_info,
                        on=['air_store_id'],
                        how='left')

del df_meta['calendar_date'], df_meta['day_of_week']
gc.collect()

In [13]:
df_meta['visit_date'] = pd.to_datetime(df_meta['visit_date'])
df_meta['weekday'] = df_meta['visit_date'].dt.weekday
df_meta['air_store_id_weekday'] = df_meta['air_store_id'].astype(str) + '_' + df_meta['weekday'].astype(str)

In [14]:
df_meta['air_store_id_holiday'] = df_meta['air_store_id'].astype(str) + '_' + df_meta['holiday_flg'].astype(str)
df_meta['air_store_id_weekday_holiday'] = df_meta['air_store_id_holiday'].astype(str) + '_' + df_meta['weekday'].astype(str)

df_meta['air_genre_name_weekday'] = df_meta['air_area_name'].astype(str) + df_meta['weekday'].astype(str)
df_meta['air_genre_name_weekday_holiday'] = df_meta['air_genre_name_weekday'].astype(str) + df_meta['holiday_flg'].astype(str)
df_meta['visitors'] = df_meta['visitors'].apply(np.log1p).values

df_meta.head()

,air_store_id,visit_date,visitors,holiday_flg,air_genre_name,air_area_name,latitude,longitude,weekday,air_store_id_weekday,air_store_id_holiday,air_store_id_weekday_holiday,air_genre_name_weekday,air_genre_name_weekday_holiday
0,air_ba937bf13d40fb24,2016-01-13,3.258097,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,air_ba937bf13d40fb24_2,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_2,Tōkyō-to Minato-ku Shibakōen2,Tōkyō-to Minato-ku Shibakōen20
1,air_ba937bf13d40fb24,2016-01-14,3.496508,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,3,air_ba937bf13d40fb24_3,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_3,Tōkyō-to Minato-ku Shibakōen3,Tōkyō-to Minato-ku Shibakōen30
2,air_ba937bf13d40fb24,2016-01-15,3.401197,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,air_ba937bf13d40fb24_4,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_4,Tōkyō-to Minato-ku Shibakōen4,Tōkyō-to Minato-ku Shibakōen40
3,air_ba937bf13d40fb24,2016-01-16,3.135494,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,5,air_ba937bf13d40fb24_5,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_5,Tōkyō-to Minato-ku Shibakōen5,Tōkyō-to Minato-ku Shibakōen50
4,air_ba937bf13d40fb24,2016-01-18,1.945910,0,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0,air_ba937bf13d40fb24_0_0,Tōkyō-to Minato-ku Shibakōen0,Tōkyō-to Minato-ku Shibakōen00


In [15]:
air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x: x.split(' ')[0]).values
air_reserve['air_store_id_visit_date'] = air_reserve['air_store_id'] + '_' + air_reserve['visit_datetime'].astype(str)

air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])

hpg_reserve['visit_date'] = hpg_reserve['visit_datetime'].apply(lambda x: x.split(' ')[0]).values

hpg_to_air_storeid = store_id_relation.set_index('hpg_store_id')
hpg_reserve['air_store_id'] = hpg_reserve['hpg_store_id'].map(hpg_to_air_storeid['air_store_id']).values
hpg_air_reserve = hpg_reserve.loc[hpg_reserve.air_store_id.notnull()]

hpg_air_reserve['air_store_id_visit_date'] = hpg_air_reserve['air_store_id'] + '_' + hpg_air_reserve['visit_date'].astype(str)
hpg_air_reserve['visit_datetime'] = pd.to_datetime(hpg_air_reserve['visit_datetime'])
hpg_air_reserve['reserve_datetime'] = pd.to_datetime(hpg_air_reserve['reserve_datetime'])

# 特征工程
## 一阶特征
### store_id的特征

1. store_id的基础信息，包括类型，地址，经纬度
2. store_id的开业天数，开业到现在的时间，最后一天有顾客到现在的时间

In [17]:
def get_storeid_fea(df_visit_data, df_store_info, df_store_id_relation):
    df_storeid = df_store_info.copy()
    
    df_storeid['air_genre_name'] = LabelEncoder().fit_transform(df_storeid['air_genre_name'].values)
    df_storeid['air_area_name'] = LabelEncoder().fit_transform(df_storeid['air_area_name'].values)
    
    first_day_dict = df_visit_data.groupby(['air_store_id'])['visit_date'].min().to_dict()
    last_day_dict = df_visit_data.groupby(['air_store_id'])['visit_date'].max().to_dict()
    run_days_dict = df_visit_data.groupby(['air_store_id'])['visit_date'].nunique().to_dict()
    
    df_storeid['first_day_to_now'] = df_storeid['air_store_id'].map(first_day_dict).values
    df_storeid['first_day_to_now'] = df_storeid['first_day_to_now'].apply(lambda x: (parse('2017-04-22') - parse(x)).days)
    
    df_storeid['last_day_to_now'] = df_storeid['air_store_id'].map(last_day_dict).values
    df_storeid['last_day_to_now'] = df_storeid['last_day_to_now'].apply(lambda x: (parse('2017-04-22') - parse(x)).days)
    
    df_storeid['run_days'] = df_storeid['air_store_id'].map(run_days_dict).values
    df_storeid['two_systems'] = df_storeid['air_store_id'].isin(df_store_id_relation['air_store_id']).values
    df_storeid['two_systems'] = df_storeid['two_systems'].astype(int)
    
    return df_storeid

In [19]:
df_storeid = get_storeid_fea(air_visit_data, air_store_info, store_id_relation)

### date的特征
1. 星期几
2. 是否节假日
3. 距离月末，月初的时间
4. 距离最近的节假日的时间

In [20]:
def get_last_next_holiday(dates, holiday_flags):
    last_holiday = []
    next_holiday = []
    len_ = len(dates)
    for i, (d, f) in enumerate(zip(dates, holiday_flags)):
        if i == 0:
            last_holiday.append(dates[i])
        else:
            # last holiday
            last_flag = 0
            for j in range(i - 1, 0, -1):
                if holiday_flags[j] == 1:
                    last_holiday.append(dates[j])
                    last_flag = 1
                    break
            if last_flag == 0:
                last_holiday.append(dates[0])
                
        # next holiday
        next_flag = 0
        for j in range(i + 1, len_):
            if holiday_flags[j] == 1:
                next_holiday.append(dates[j])
                next_flag = 1
                break
        if next_flag == 0:
            next_holiday.append(dates[-1])
    return last_holiday, next_holiday

In [21]:
def get_days_to_month_end(x):
    month = int(x.split('-')[1])
    day = int(x.split('-')[2])
    if month in [1, 3, 5, 7, 8, 10, 12]:
        return 31 - day
    elif month in [2]:
        return 28 - day
    else:
        return 30 - day

In [22]:
def get_time_fea(df_date_info):
    df_date = df_date_info.copy()
    df_date['day_to_month_end'] = df_date['calendar_date'].apply(lambda x: get_days_to_month_end(x)).values
    df_date['day_to_month_start'] = df_date['calendar_date'].apply(lambda x: int(x.split('-')[0])).values
    
    df_date['day_of_week'] = df_date['calendar_date'].apply(lambda x: parse(x).weekday()).values
    df_date['is_weekend'] = df_date['day_of_week'].apply(lambda x: x > 4).values
    
    last_holiday, next_holiday = get_last_next_holiday(df_date['calendar_date'].values, df_date['holiday_flg'].values)
    df_date['last_holiday'] = last_holiday
    df_date['next_holiday'] = next_holiday

    df_date['day_to_last_holiday'] = (df_date['calendar_date'].apply(lambda x: parse(x))
                                      - df_date['last_holiday'].apply(lambda x: parse(x)))
    df_date['day_to_last_holiday'] = df_date['day_to_last_holiday'].apply(lambda x: x.days).values
    
    df_date['day_to_next_holiday'] = (df_date['next_holiday'].apply(lambda x: parse(x))
                                      - df_date['calendar_date'].apply(lambda x: parse(x)))
    df_date['day_to_next_holiday'] = df_date['day_to_next_holiday'].apply(lambda x: x.days).values
    
    df_date.rename(columns={'calendar_date': 'visit_date'}, inplace=True)
    
    df_date['visit_date'] = pd.to_datetime(df_date['visit_date'])
    for i in [3, 2, 1, -1]:
        date_info_temp = df_date.copy()
        date_info_temp['visit_date'] = date_info_temp['visit_date'].apply(lambda x: x + timedelta(i))
        date_info_temp.rename(columns={'holiday_flg': 'ahead_holiday_{}'.format(i)},
                              inplace=True)
        
        df_date = df_date.merge(date_info_temp[['visit_date', 'ahead_holiday_{}'.format(i)]],
                                on=['visit_date'],
                                how='left')
    del df_date['last_holiday'], df_date['next_holiday']
    gc.collect()
    return df_date

In [23]:
df_date = get_time_fea(date_info)

## 二阶特征：store_id & day

因为day这个东西只有一个,而且未来和之前的是不一致的,那么做特征的时候就需要找day相关的属性,同样的store_id的属性也可以构建特征

### store_id+day(related特征/holiday/weekday)

In [24]:
def get_st_date(en_date, days):
    # st_date <= date < end_date
    st_date = en_date - timedelta(days)
    return st_date

def get_label_st_date(st_date, days):
    # st_date <= date < end_date
    end_date = st_date + timedelta(days)
    return st_date, end_date

def get_df_label(df, date, days=39, is_te=False):
    st_date, en_date = get_label_st_date(date, days)
    print('label date, ', st_date, en_date)
    ind1 = df['visit_date'] < en_date
    ind2 = df['visit_date'] >= st_date
    if is_te:
        return df.loc[(ind2), ['air_store_id', 'visit_date', 'air_store_id_holiday',
                               'air_genre_name', 'air_genre_name_weekday', 'air_genre_name_weekday_holiday']].copy()
    return df.loc[(ind1 & ind2), ['air_store_id', 'visit_date', 'air_store_id_holiday',
                                  'air_genre_name', 'air_genre_name_weekday', 'air_genre_name_weekday_holiday', 'visitors']].copy()

### 特征工程1：visitors相关特征
#### air_store_id+visitors特征（recent days）

最近$N(N=14,28,56,1000)$天air_store_id的统计特征

In [35]:
def get_store_id_visitors_sts_features(df, en_date, key='air_store_id', date_col='visit_date',
                                       days_list=[1000, 56, 28, 14]):
    t0 = time()
    df_features = pd.DataFrame()
    df_features[key] = df_features[key].unique()
    
    df_tmp = df.copy()
    for days in days_list:
        st_date = get_st_date(en_date, days)
        df_tmp = df_tmp.loc[((df[date_col] >= st_date) & (df[date_col] < en_date))].copy()
        df_features_tmp = df_tmp.groupby([key], as_index=False)['visitors'].agg({'{}_visitors_min_{}'.format(keys, days): 'min'
                                                                                 '{}_visitors_mean_{}'.format(key, days): 'mean',
                                                                                 '{}_visitors_sum_{}'.format(key, days): 'sum',
                                                                                 '{}_visitors_median_{}'.format(key, days): 'median',
                                                                                 '{}_visitors_max_{}'.format(key, days): 'max',
                                                                                 '{}_visitors_count_{}'.format(key, days): 'count',
                                                                                 '{}_visitors_std_{}'.format(key, days): 'std',
                                                                                 '{}_visitors_quantile_{}'.format(key, days): 'quantile',
                                                                                 '{}_visitors_skew_{}'.format(key, days): 'skew'})
        df_features = df_features.merge(df_features_tmp,
                                        on=key,
                                        how='left')
        del df_features_tmp
        gc.collect()
    print('For store id sts features, we spend {} seconds.'.format(time() - t0))
    return df_features